# MongoDB - join

MongoDB is een document database.

MongoDB:

* document: zelf-beschrijvende gestructureerde data
    * met mogelijk samengestelde waarden
    * met mogelijk meerwaardige waarden
    * ook: combineren meerwaardig en samengesteld
* collection: verzameling "gelijksoortige" documenten

*Hoe vertaal je een datamodel naar een MongoDB-database?*

![persoon-event model](persoon-event-relatie.png)

Voorbeeldmodel

* personen
    * elke persoon neemt mogelijk deel in meerdere events
* events
    * elk event heeft mogelijk meerdere deelnemers (personen)
* relatie tussen personen en events?

In SQL: apart relatie-object nodig ("deelnemers" o.i.d.)

In MongoDB: inbedden of verwijzen (embedding or referencing)

Inbedden van persoon-documenten bij events???
```
 {'_id': 2, 'datum': '2020-12-13', 'locatie': 'Breda',
  'deelnemers': [{'_id': 5, 'naam': 'Anna Bouma', 
                  'email': 'anna.bouma@fmail.com', 
                  'plaats': 'Breda'},
                 {'_id': 3, 'naam': 'Frans Gerrits', 
                  'email': 'fg23@mymail.net', 
                  'plaats': 'Tilburg'},
                 {'_id': 4, 'naam': 'Joke Boer', 
                  'email': 'jboer@farmer.net',
                  'plaats': 'Breda'},
                 {'_id': 2, 'naam': 'Erik Bakker', 
                  'email': 'erikB@ziggomail.nl', 
                  'plaats': 'Tilburg'}
                ]}
```

Inbedden van event-documenten bij persoon???
```
 {'_id': 4,
  'naam': 'Joke Boer',
  'email': 'jboer@farmer.net',
  'plaats': 'Breda',
  'events': [{'_id': 2, 'datum': '2020-12-13', 
              'locatie': 'Breda'}, 
             {'_id': 5, 'datum': '2020-12-23', 
              'locatie': 'Breda'}]}
  
```  

Gebruik van *verwijzingen* (foreign keys) in persoon-document:

```
{'_id': 4,
  'naam': 'Joke Boer',
  'email': 'jboer@farmer.net',
  'plaats': 'Breda',
  'events': [2, 5]}
```

*Vraag: waarom is dit handiger dan omgekeerd?*

In SQL aparte tabel `deelname` nodig -

met foreign keys van `persoon` en `event`. 
    
<img src="deelname-tabel.png" width="150">

In een relationele database heb je veel minder mogelijkheden voor embedding,
omdat je geen meervoudige waarden kunt hebben.

Embedding hoeft niet noodzakelijk tot een niet-redundante representatie te leiden:
het bovenstaande voorbeeld kun je nog steeds als "genormaliseerd" beschouwen.

Verwijzen-voorbeeld

(personen-aanmeldingen-wedstrijden)
(voorbeeld: aanmeldingen als aparte documenten)

NB: in beide gevallen zijn wedstrijden aparte documenten

Maak entiteit apart document als

* deze afzonderlijk aangemaakt en bijgewerkt wordt;
* en deze afzonderlijk opgevraagd wordt.

Voorbeeld: aanmeldingen bijwerken: altijd persoonsgegevens nodig => embedden

Dit is een voorbeeld van de manier waarop in MongoDB het gebruik (de queries) centraal staan, veel meer dan de data.

Join: combineren van documenten

* uitprogrammeren in toepassing, of:
* join (`lookup`) als onderdeel van *aggregation pipeline*

MongoDB `aggregate` pijplijn:

```
collection.aggregate([ {... stap 1 ...},
                       {... stap 2 ...},
                       ...,
                       {... stap N ...}])
```
stap 1: werkt op alle documenten in `collection`

stap2: werkt op de resultaten van stap 1,

enz...

Functies voor `aggregate` pijplijn:
    
* `$match` - filteren van documenten
* `$group` - groeperen en samenvatten (tellen enz.)
* `$project` - filteren van velden
* `$lookup` - combineren met andere collection ("join")

In [ ]:
import os
os.environ["PATH"]=os.environ["PATH"] + ":/usr/local/bin"

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo

pd.set_option('max_colwidth',160)

userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
print("Database name: " + dbname)

print('Mongo version', pymongo.__version__)
client = pymongo.MongoClient('localhost', 27017)
db = client[dbname]

In [ ]:
persons = db.persons
persons.drop()
os.system('mongoimport -d ' + dbname + ' -c persons persons.json')

In [ ]:
list(db.persons.find())

In [ ]:
events = db.events
events.drop()
os.system('mongoimport -d ' + dbname + ' -c events events.json')

In [ ]:
list(db.events.find())

Samenvatten: aantal personen per woonplaats

In [ ]:
cursor = persons.aggregate([{"$group":{"_id": "$plaats", "aantal": {"$sum": 1} }}])
list(cursor)

Lookup: combineren van personen en events
Deelnemers van event op `2020-12-13`

In [ ]:
cursor = events.aggregate([{"$match": {"datum": "2020-12-13"}},
                          ])
list(cursor)

In [ ]:
cursor = events.aggregate([{"$match": {"datum": "2020-12-13"}},
                           {"$lookup":{"from": "persons", "localField": "_id", "foreignField": "events",
                                        "as": "deelnemers"}}])
list(cursor)

In [ ]:
cursor = events.aggregate([{"$match": {"datum": "2020-12-13"}},
                           {"$lookup":{"from": "persons", "localField": "_id", "foreignField": "events",
                                        "as": "deelnemers"}},
                           {"$project": {"_id": 0, "deelnemers":1}},
                           {"$project": {"deelnemers._id":0, "deelnemers.events": 0}}])
list(cursor)

In [ ]:
cursor = events.aggregate([{"$match": {"datum": "2020-12-13"}},
                           {"$lookup":{"from": "persons", "localField": "_id", "foreignField": "events",
                                        "as": "deelnemers"}},
                           {"$project": {"_id": 0, "deelnemers":1}},
                           {"$project": {"deelnemers._id":0, "deelnemers.events": 0}},
                           {"$unwind": "$deelnemers"},
                           {"$replaceRoot": {"newRoot": "$deelnemers"}}])
list(cursor)

Opdrachten: 

* maak een overzicht van het aantal events per locatie.
* maak lijst van events van Linda Fransen.